In [93]:
#Variables that contains the user credentials to access Twitter API
access_token = ""
access_token_secret = ""
consumer_key = ""
consumer_secret = ""

import os
import sys
import json
import time
import math
from tweepy import Cursor
import tweepy
from tweepy import OAuthHandler
import networkx as nx
import pprint 
pp = pprint.PrettyPrinter(indent=3)
from operator import itemgetter

# authentication
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Scrape Friends of SamsungMobile

In [45]:
def paginate(items, n):
    """Generate n-sized chunks from items"""
    for i in range(0, len(items), n):
        yield items[i:i+n]

In [3]:
MAX_FRIENDS = 15000

screen_name = "SamsungMobile" #enter screen name on who you would like to check on it.
print("collecting data for " + screen_name)
dirname = "output/users2/{}".format(screen_name)
max_pages = math.ceil(MAX_FRIENDS / 5000)
try:
    os.makedirs(dirname, mode=0o755, exist_ok=True)
except OSError:
    print("Directory {} already exists".format(dirname))
except Exception as e:
    print("Error while creating directory {}".format(dirname))
    print(e)
    sys.exit(1)
               
# get friends for a given user
fname = "output/users2/{}/friends.json".format(screen_name)
with open(fname, 'w') as f:
    for friends in Cursor(api.friends_ids, screen_name=screen_name).pages(max_pages):
        for chunk in paginate(friends, 100):
            users = api.lookup_users(user_ids=chunk)
            for user in users:
                f.write(json.dumps(user._json)+"\n")
        if len(friends) == 5000:
            print("More results available. Sleeping for 60 seconds to avoid rate limit")
            time.sleep(60)
print("task completed")

collecting data for SamsungMobile
task completed


# Scrape Friends of Friends 

In [69]:
screen_name = "SamsungMobile"
friends_file = 'output/users2/{}/friends.json'.format(screen_name)
with open(friends_file) as f2:
       
    friends = []
    for line in f2:
        profile = json.loads(line)
        friends.append(profile['screen_name'])
        
friends[423]

'Veronica'

In [94]:
MAX_FRIENDS = 15000

#Use Counter to specify which friends to start scraping from due to rate limit error
counter = 0

for fr in friends:
    if counter >= 423:
        screen_name = fr #enter screen name on who you would like to check on it.
        print("collecting data for " + screen_name)
        dirname = "output/users2/SamsungMobile/{}".format(screen_name)
        max_pages = math.ceil(MAX_FRIENDS / 5000)
        try:
            os.makedirs(dirname, mode=0o755, exist_ok=True)
        except OSError:
            print("Directory {} already exists".format(dirname))
        except Exception as e:
            print("Error while creating directory {}".format(dirname))
            print(e)
            sys.exit(1)

        # get friends for a given user
        fname = "output/users2/SamsungMobile/{}/friends.json".format(screen_name)
        try:
            with open(fname, 'w') as f:
                for friends in Cursor(api.friends_ids, screen_name=screen_name).pages(max_pages):
                    for chunk in paginate(friends, 100):
                        users = api.lookup_users(user_ids=chunk)
                        for user in users:
                            f.write(json.dumps(user._json)+"\n")
                    if len(friends) == 5000:
                        print("More results available. Sleeping for 60 seconds to avoid rate limit")
                        time.sleep(60)
            print("task completed")
            print("Index: ", counter)
            
        except TweepError:
            print('Missed: ', screen_name)
        except RateLimitError:
            print('Rate limit error')
    counter += 1 

# Generate Node and Edge 

In [97]:
filter1 = 50
filter2 = 100

import sys
import json
from random import sample

if __name__ == '__main__':
    screen_name = "SamsungMobile"
    
    master_nodelist = []
    master_edgelist = []
    
    friends_file = 'output/users2/{}/friends.json'.format(screen_name)
    with open(friends_file) as f2:
       
        friends = []
        for line in f2:
            profile = json.loads(line)
            friends.append(profile['screen_name'])
            
            
    #Filter out junk friends
    tempfriends = []
    for fr in friends:
        fof_file = 'output/users2/SamsungMobile/{}/friends.json'.format(fr)    
        try:
            with open(fof_file) as f2:
                tempfriends.append(fr)
        except:
            print("Missed: ", fr)    
    
    #Add to master edge and node list 
    master_nodelist.append('SamsungMobile')
    
    samsung_fof = {}
    
    for fr in tempfriends[:filter1]:
        try:
            fof_file = 'output/users2/SamsungMobile/{}/friends.json'.format(fr)
            
            master_nodelist.append(fr)
            master_edgelist.append(('SamsungMobile',fr))
            with open(fof_file) as f2:
                frlist = []
                count = 0 
                for line in f2:
                    if count <= filter2:
                        profile = json.loads(line)
                        frlist.append(profile['screen_name'])
                        count += 1
                    else:
                        continue
                samsung_fof[fr] = frlist
            
        except:
            print("Missed: ", fr)  #Missed friends due to their account being protected

Missed:  VineCreators
Missed:  CokeZero
Missed:  vd9520
Missed:  SamsungMobileUK
Missed:  gamespotuk
Missed:  liamgh
Missed:  CNETUK


Second Level Node and Edge 

In [98]:
for fr in friends:
    try:
        for fr2 in samsung_fof[fr]:
            if fr2 not in master_nodelist:
                master_nodelist.append(fr2)
    except:
        print("Missed: ", fr)
                
for fr in friends:
    try:
        for fr2 in samsung_fof[fr]:
            master_edgelist.append((fr,fr2))
    except:
        print("Missed: ", fr) #Missed friends are due to their account being protected

Missed:  VineCreators
Missed:  CokeZero
Missed:  claudinehesse
Missed:  JPatrelle
Missed:  lambeajulo
Missed:  SBelouezzane
Missed:  WeWoreWhat
Missed:  Eug_enie
Missed:  SheenaSmith_
Missed:  PriyankaKapadia
Missed:  Luca_Lanzoni
Missed:  marchettisimone
Missed:  mmmargherita
Missed:  KissesInPINK_
Missed:  nosleepteam24
Missed:  AntiaHanoz
Missed:  PatrickRaham
Missed:  vishmon2012
Missed:  vd9520
Missed:  VinnyDuke
Missed:  JohnathanAllan_
Missed:  AkdotIk
Missed:  CubanSunset
Missed:  elvenage
Missed:  mjfan010
Missed:  om
Missed:  engadgetmobile
Missed:  SamsungDevEU
Missed:  rudymancuso
Missed:  Shonduras
Missed:  SamMobiles
Missed:  SamsungEC
Missed:  CNETNews
Missed:  SamsungCTC
Missed:  ZDNet
Missed:  SamsungPY
Missed:  googlenexus
Missed:  Samsung
Missed:  harpersbazaarus
Missed:  VVFriedman
Missed:  DieselBlackGold
Missed:  michaelhoeweler
Missed:  FashionWeek
Missed:  NikkiElizDemere
Missed:  montblanc_world
Missed:  swarovski
Missed:  johnpoz
Missed:  Twexlist
Missed:  Ave

In [99]:
G = nx.Graph()

G.add_nodes_from(master_nodelist)
G.add_edges_from(master_edgelist)

In [100]:
import community
import matplotlib.pyplot as plt

In [101]:
%matplotlib qt

In [102]:
#Based on Louvain heuristices
#https://perso.uclouvain.be/vincent.blondel/research/louvain.html
part = community.best_partition(G)


# Color mapping using community structure
values = [part.get(node) for node in G.nodes()]
#values = [mod.get(node) for node in G.nodes()]
d = dict(nx.degree(G))

#https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.drawing.nx_pylab.draw_networkx.html
#https://matplotlib.org/examples/color/colormaps_reference.html
nx.draw_spring(G, cmap=plt.get_cmap('Pastel1'), node_color = values, node_size=70, with_labels=True, edge_color='grey')
plt.show()

C:\Users\Vincent\Miniconda3\lib\site-packages\networkx\drawing\nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


# Number of Nodes/Edges

In [103]:
print('Nodes: ',G.number_of_nodes())
print('Edges: ',G.number_of_edges())

Nodes:  4423
Edges:  4648


# Density

In [104]:
density = nx.density(G)
print("Network density:", density)

Network density: 0.0004752919266941964


# Degree Centrality

In [105]:
degree_dict = dict(G.degree(G.nodes()))
#pp.pprint(degree_dict)
nx.set_node_attributes(G, name ='degree', values = degree_dict)

sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)
sorted_degree[:5]

[('TwitterVideo', 107),
 ('MKBHD', 104),
 ('SamsungMobileUS', 103),
 ('SamsungAU', 103),
 ('ChrisEvans', 103)]

# Declaring Centralities

In [107]:
betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(G, name = 'betweenness', values = betweenness_dict)
nx.set_node_attributes(G, name = 'eigenvector', values = eigenvector_dict)

# Eigenvector Centrality

In [108]:
nx.set_node_attributes(G, part, 'modularity')
# First get a list of just the nodes in that class
class0 = [n for n in G.nodes() if G.node[n]['modularity'] == 5]

# Then create a dictionary of the eigenvector centralities of those nodes
class0_eigenvector = {n:G.node[n]['eigenvector'] for n in class0}

# Then sort that dictionary and print the first 5 results
class0_sorted_by_eigenvector = sorted(class0_eigenvector.items(), key=itemgetter(1), reverse=True)

print("Modularity Class 0 Sorted by Eigenvector Centrality:")
for node in class0_sorted_by_eigenvector:
    print("Name:", node[0], "| Eigenvector Centrality:", node[1])

Modularity Class 0 Sorted by Eigenvector Centrality:
Name: Shelley__Watts | Eigenvector Centrality: 0.06745701629295707
Name: wada_ama | Eigenvector Centrality: 0.010107035566169134
Name: WRugbyLeague | Eigenvector Centrality: 0.005126550217072168
Name: JaredTallent | Eigenvector Centrality: 0.005126550217072168
Name: SweetSciencePhD | Eigenvector Centrality: 0.005126550217072168
Name: IndigMaraProjct | Eigenvector Centrality: 0.005126550217072168
Name: ElsieSeriat | Eigenvector Centrality: 0.005126550217072168
Name: StrongerSmarter | Eigenvector Centrality: 0.005126550217072168
Name: chrissarra | Eigenvector Centrality: 0.005126550217072168
Name: deek207 | Eigenvector Centrality: 0.005126550217072168
Name: mjaltamura | Eigenvector Centrality: 0.005126550217072168
Name: ModBoxau | Eigenvector Centrality: 0.005126550217072168
Name: trboxing | Eigenvector Centrality: 0.005126550217072168
Name: ChampsOlympics | Eigenvector Centrality: 0.005126550217072168
Name: TagzartDesign | Eigenvector

In [109]:
nx.set_node_attributes(G, part, 'modularity')
# First get a list of just the nodes in that class
class1 = [n for n in G.nodes() if G.node[n]['modularity'] == 1]

# Then create a dictionary of the eigenvector centralities of those nodes
class1_eigenvector = {n:G.node[n]['eigenvector'] for n in class1}

# Then sort that dictionary and print the first 5 results
class1_sorted_by_eigenvector = sorted(class1_eigenvector.items(), key=itemgetter(1), reverse=True)

print("Modularity Class 1 Sorted by Eigenvector Centrality:")
for node in class1_sorted_by_eigenvector:
    print("Name:", node[0], "| Eigenvector Centrality:", node[1])

Modularity Class 1 Sorted by Eigenvector Centrality:
Name: TwitterBusiness | Eigenvector Centrality: 0.07263638954489661
Name: wbgames | Eigenvector Centrality: 0.012514122787930914
Name: DetPikachuMovie | Eigenvector Centrality: 0.011215904010570597
Name: ShilohNoelle | Eigenvector Centrality: 0.005520003699711186
Name: michneid1 | Eigenvector Centrality: 0.005520003699711186
Name: BloombergDotOrg | Eigenvector Centrality: 0.005520003699711186
Name: MikeBloomberg | Eigenvector Centrality: 0.005520003699711186
Name: TheTerminal | Eigenvector Centrality: 0.005520003699711186
Name: Bloomberg | Eigenvector Centrality: 0.005520003699711186
Name: blaw | Eigenvector Centrality: 0.005520003699711186
Name: business | Eigenvector Centrality: 0.005520003699711186
Name: magancrane | Eigenvector Centrality: 0.005520003699711186
Name: KarenMahabir | Eigenvector Centrality: 0.005520003699711186
Name: ldkeith | Eigenvector Centrality: 0.005520003699711186
Name: Enstereo | Eigenvector Centrality: 0.00

In [110]:
sorted_eigenvector = sorted(eigenvector_dict.items(), key=itemgetter(1), reverse=True)

n = 5
print("Top {} nodes by eigenvector centrality:".format(n))
for b in sorted_eigenvector[:n]:
    print(b)

Top 5 nodes by eigenvector centrality:
('SamsungMobile', 0.35431199325394636)
('jeffweiner', 0.2437962559699591)
('drewhouston', 0.20886572544504112)
('marissamayer', 0.18349489024243165)
('TwitterVideo', 0.1763754436933672)


# Betweeness Centrality

In [111]:
sorted_betweenness = sorted(betweenness_dict.items(), key=itemgetter(1), reverse=True)

n = 5
print("Top {} nodes by betweenness centrality:".format(n))
for b in sorted_betweenness[:n]:
    print(b)

Top 5 nodes by betweenness centrality:
('SamsungMobile', 0.8939297420596047)
('TwitterVideo', 0.04892549749683238)
('JChengWSJ', 0.04516405449874274)
('SamsungMobileKE', 0.04516405449874274)
('Lil_Eazy_Ana_42', 0.044935568106188685)


# Generate Subgraph

In [117]:
targetcom = [0, 26, 27, 29, 5]
subgra = []

for x in part:
    if part[x] in targetcom:
        subgra.append(x)

subgraph = G.subgraph(subgra)

values2 = [part.get(node) for node in subgraph]

In [118]:
nx.draw_spring(subgraph, cmap=plt.get_cmap('Pastel1'), with_labels=True, node_color = values2)

C:\Users\Vincent\Miniconda3\lib\site-packages\networkx\drawing\nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


# Testing Area (Own Reference)

In [112]:
target = 'Techmeme'
print('eigenvector: ', eigenvector_dict[target])
print('betweeness: ', betweenness_dict[target])
print('degree: ', degree_dict[target])

eigenvector:  0.03051270188089184
betweeness:  0.0005895272683537223
degree:  3


In [114]:
nodetarget = 'SamsungMobile'

for tupley in master_edgelist:
    if (tupley[0]==nodetarget) or (tupley[1]==nodetarget):
        print('Connected Nodes are ', tupley)

Connected Nodes are  ('SamsungMobile', 'TwitterBusiness')
Connected Nodes are  ('SamsungMobile', 'TwitterVideo')
Connected Nodes are  ('SamsungMobile', 'Olympics')
Connected Nodes are  ('SamsungMobile', 'TomDaley1994')
Connected Nodes are  ('SamsungMobile', 'Shelley__Watts')
Connected Nodes are  ('SamsungMobile', 'QuentinDOR')
Connected Nodes are  ('SamsungMobile', 'peterwells')
Connected Nodes are  ('SamsungMobile', 'askmartysmarty')
Connected Nodes are  ('SamsungMobile', 'DjPruluv')
Connected Nodes are  ('SamsungMobile', 'sharonprofis')
Connected Nodes are  ('SamsungMobile', 'TypicalGamer')
Connected Nodes are  ('SamsungMobile', 'ProSyndicate')
Connected Nodes are  ('SamsungMobile', 'iHasCupquake')
Connected Nodes are  ('SamsungMobile', 'deGRISOGONO')
Connected Nodes are  ('SamsungMobile', 'AndyBoxall')
Connected Nodes are  ('SamsungMobile', 'Snoopy')
Connected Nodes are  ('SamsungMobile', 'CES')
Connected Nodes are  ('SamsungMobile', 'AndroidAuth')
Connected Nodes are  ('SamsungMobi

In [115]:
nodetarget = 'MKBHD'

for tupley in master_edgelist:
    if (tupley[0]==nodetarget) or (tupley[1]==nodetarget):
        print('Connected Nodes are ', tupley)

Connected Nodes are  ('SamsungMobile', 'MKBHD')
Connected Nodes are  ('ProSyndicate', 'MKBHD')
Connected Nodes are  ('CES', 'MKBHD')
Connected Nodes are  ('MKBHD', 'Eminem')
Connected Nodes are  ('MKBHD', 'LikeTeslaKim')
Connected Nodes are  ('MKBHD', 'TickTickTeam')
Connected Nodes are  ('MKBHD', 'betablock3r')
Connected Nodes are  ('MKBHD', 'BillGates')
Connected Nodes are  ('MKBHD', 'rtingsdotcom')
Connected Nodes are  ('MKBHD', 'willduwilldu')
Connected Nodes are  ('MKBHD', 'gracemburke')
Connected Nodes are  ('MKBHD', 'NBT88YT')
Connected Nodes are  ('MKBHD', 'PixelFilmStudio')
Connected Nodes are  ('MKBHD', 'LumaForge')
Connected Nodes are  ('MKBHD', 'Erdayastronaut')
Connected Nodes are  ('MKBHD', 'geekyranjit')
Connected Nodes are  ('MKBHD', 'Co_Mill')
Connected Nodes are  ('MKBHD', 'DougDeMuro')
Connected Nodes are  ('MKBHD', 'jacksonhvisuals')
Connected Nodes are  ('MKBHD', 'BenSullins')
Connected Nodes are  ('MKBHD', 'ronnychieng')
Connected Nodes are  ('MKBHD', 'CONTEMPORIS

In [116]:
targetcom = [0]

for x in part:
    if part[x] in targetcom:
        print(x)

SamsungMobile
askmartysmarty
Samsung_IN
susiebubble
8bitfiction
scoutmagph
MobileLegendsOL
SilakboPH
LiveSmart
circuitbreaker
HuffPostTech
iflixletsplay
DigitalTrends
VergeScience
iflixph
kir7a
vergepop
Snowden
NicheDumlao
Spotdotph
bookymanila
meiyah
technobaboy
Ning4u
officialPTB
ShutangInaBeks
showbiznest
lush_angel
ctctweets
Apple
inversedotcom
PokemonGoApp
Verniecenciso
adamparnala
marcogumabao
richardjuan
macworld
Google
Gizmodo
engadget
WIRED
verge
mashable
TechCrunch
Samsung
abhishek
amitbhawani
nair_rinku
BGRIndia
afaqs
shoutmeloud
NothingWired
devils_workshop
Save_Delete
labnol
TechHive
GA
WSJdigits
socialgood
AllTechSecrets
NBCNewsMACH
Techmeme
CNET
Scobleizer
anildash
WIREDBusiness
ForbesTech
TEDTalks
guardiantech
CNETNews
nikhilchawla
buffer
TheUnbiasedBlog
SamsungMobileIN
